# Polars Avanzado

En este notebook exploraremos características avanzadas de Polars que te permitirán escribir código de análisis de datos profesional, eficiente y escalable:

* **Lazy Evaluation profunda:** Optimización de consultas
* **Window Functions:** Operaciones analíticas complejas
* **Joins optimizados:** Fusión eficiente de DataFrames
* **Pivoting y Crosstabs:** Transformación de datos
* **I/O de alto rendimiento:** Lectura/escritura optimizada
* **Decisiones arquitectónicas:** Cuándo usar Polars, Pandas o Dask

In [ ]:
import polars as pl
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

print(f"Polars {pl.__version__}")

## Lazy Evaluation Profunda

La **evaluación lazy** es el corazón de la eficiencia de Polars. Las operaciones no se ejecutan inmediatamente, sino que se construye un plan de ejecución optimizado.

In [ ]:
# Crear datos de ejemplo
df = pl.DataFrame({
    'id': range(1, 101),
    'nombre': [f'persona_{i}' for i in range(100)],
    'departamento': ['Ventas', 'IT', 'HR', 'Finanzas'] * 25,
    'salario': np.random.randint(40000, 120000, 100),
    'fecha_contrato': [datetime(2020, 1, 1) + timedelta(days=x) for x in range(100)]
})

print(f"DataFrame shape: {df.shape}")
print(df.head())

In [ ]:
# Construcción de query lazy
query = (df
    .lazy()
    .filter(pl.col('salario') > 60000)
    .select(['nombre', 'departamento', 'salario'])
    .sort('salario', descending=True)
)

# Aún no se ejecutó nada
print(f"Tipo: {type(query)}")
print(f"Es lazy: {isinstance(query, pl.LazyFrame)}")

In [ ]:
# Ver el plan de ejecución (antes de optimize)
print("Plan sin optimizar:")
print(query.explain(optimized=False))

In [ ]:
# Ver el plan optimizado
print("Plan optimizado (lo que realmente ejecuta):")
print(query.explain(optimized=True))

In [ ]:
# Ejecutar la query
result = query.collect()
print(f"Resultado ({len(result)} filas):")
print(result)

### Beneficios de Lazy Evaluation:

1. **Optimización automática:** Predicados push-down, eliminación de columnas no usadas
2. **Memoria eficiente:** Solo materializa lo necesario
3. **Paralelización:** Automática en múltiples núcleos
4. **Simplificación:** Código más legible

## Window Functions

Las **funciones de ventana (window functions)** permiten cálculos analíticos complejos sobre grupos de filas.

In [ ]:
# Crear datos de ventas
ventas = pl.DataFrame({
    'fecha': ['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04',
              '2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04'] * 3,
    'producto': ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B',
                 'C', 'C', 'C', 'C', 'A', 'A', 'A', 'A',
                 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C'],
    'monto': [100, 150, 120, 180, 200, 250, 300, 280,
              75, 95, 110, 130, 110, 160, 130, 190,
              210, 260, 310, 290, 85, 105, 120, 140]
})

print(f"Ventas shape: {ventas.shape}")
print(ventas.head())

In [ ]:
# Window function: suma acumulada por producto
resultado = ventas.with_columns(
    pl.col('monto').cum_sum().over('producto').alias('suma_acumulada')
)

print("Con suma acumulada por producto:")
print(resultado.sort(['producto', 'fecha']))

In [ ]:
# Window functions: rank y row_number
resultado = ventas.with_columns(
    pl.col('monto').rank(method='dense').over('producto').alias('rank_monto'),
    pl.col('monto').rank(method='ordinal').over('producto').alias('row_number')
)

print("Con ranking por producto:")
print(resultado.sort(['producto', 'monto'], descending=[False, True]))

In [ ]:
# Lag y Lead: acceder a filas anteriores/siguientes
resultado = ventas.with_columns(
    pl.col('monto').lag().over('producto').alias('monto_anterior'),
    pl.col('monto').lead().over('producto').alias('monto_siguiente')
)

print("Con lag/lead por producto:")
print(resultado.sort(['producto', 'fecha']))

## Joins Optimizados

Polars realiza joins de forma muy eficiente, especialmente con evaluación lazy.

In [ ]:
# Crear dos DataFrames para unir
empleados = pl.DataFrame({
    'empleado_id': [1, 2, 3, 4, 5],
    'nombre': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
    'departamento': ['Ventas', 'IT', 'HR', 'Ventas', 'IT']
})

salarios = pl.DataFrame({
    'empleado_id': [1, 2, 3, 4, 5],
    'salario': [50000, 80000, 45000, 55000, 90000],
    'bonus': [5000, 8000, 0, 5500, 9000]
})

print("Empleados:")
print(empleados)
print("\nSalarios:")
print(salarios)

In [ ]:
# Inner join
resultado_inner = empleados.join(
    salarios,
    on='empleado_id',
    how='inner'
)

print("Inner Join:")
print(resultado_inner)

In [ ]:
# Left join con múltiples columnas de join
# Crear datos con más columnas de unión
tabla_a = pl.DataFrame({
    'ano': [2023, 2023, 2024, 2024],
    'trimestre': [1, 2, 1, 2],
    'ventas': [100, 150, 120, 180]
})

tabla_b = pl.DataFrame({
    'ano': [2023, 2023, 2024, 2024],
    'trimestre': [1, 2, 1, 2],
    'gastos': [60, 80, 70, 100]
})

resultado = tabla_a.join(
    tabla_b,
    on=['ano', 'trimestre'],
    how='inner'
)

print("Join en múltiples columnas:")
print(resultado)
print("\nCon columna calculada:")
resultado = resultado.with_columns(
    (pl.col('ventas') - pl.col('gastos')).alias('ganancia')
)
print(resultado)

## Pivoting y Crosstabs

Transformar datos de formato largo a ancho.

In [ ]:
# Crear datos en formato largo
datos_largo = pl.DataFrame({
    'ano': [2023, 2023, 2023, 2023, 2024, 2024, 2024, 2024],
    'trimestre': ['Q1', 'Q2', 'Q3', 'Q4', 'Q1', 'Q2', 'Q3', 'Q4'],
    'metrica': ['ventas', 'ventas', 'ventas', 'ventas', 'ventas', 'ventas', 'ventas', 'ventas'],
    'departamento': ['A', 'A', 'B', 'B', 'A', 'A', 'B', 'B'],
    'valor': [100, 150, 200, 180, 120, 160, 210, 190]
})

print("Datos en formato largo:")
print(datos_largo)

In [ ]:
# Pivot: convertir a formato ancho
datos_ancho = datos_largo.pivot(
    index=['ano', 'departamento'],
    columns='trimestre',
    values='valor',
    aggregate_function='first'
)

print("Datos en formato ancho:")
print(datos_ancho)

In [ ]:
# Unpivot: convertir de ancho a largo
datos_reconvertidos = datos_ancho.unpivot(
    index=['ano', 'departamento'],
    variable_name='trimestre',
    value_name='valor'
)

print("Reconvertido a formato largo:")
print(datos_reconvertidos.sort(['ano', 'departamento', 'trimestre']))

## I/O de Alto Rendimiento

Polars es muy eficiente en la lectura y escritura de datos.

In [ ]:
# Crear datos de ejemplo
datos_grandes = pl.DataFrame({
    'id': range(10000),
    'valor': np.random.rand(10000),
    'categoria': np.random.choice(['A', 'B', 'C'], 10000)
})

# Escribir en Parquet con compresión
datos_grandes.write_parquet('datos_optimizado.parquet', compression='zstd')
print("✓ Guardado en Parquet con compresión zstd")

# Leer de forma lazy (solo metadatos)
df_lazy = pl.scan_parquet('datos_optimizado.parquet')
print(f"\nLazyFrame escaneado: {type(df_lazy)}")

# Ejecutar con filtro (solo se leen las filas necesarias)
resultado = df_lazy.filter(pl.col('valor') > 0.9).collect()
print(f"Filas con valor > 0.9: {len(resultado)}")

In [ ]:
# Lectura particionada (para archivos grandes divididos en carpetas)
# Guardar con particiones
datos_grandes.write_parquet(
    'datos_particionados/',
    partition_by='categoria'
)
print("✓ Guardado en Parquet particionado por categoría")

# Leer datos particionados
df_particionado = pl.scan_parquet('datos_particionados/*.parquet')
resultado = df_particionado.filter(pl.col('categoria') == 'A').collect()
print(f"Filas en categoría A: {len(resultado)}")

## Decisiones Arquitectónicas: Polars vs Pandas vs Dask

### Matriz de Decisión

| Situación | Recomendación | Razón |
| :--- | :--- | :--- |
| Datos < 1GB | **Pandas** | Suficientemente rápido, ecosistema amplio |
| Datos 1-20GB | **Polars** | Rápido, eficiente, API moderna |
| Datos > 20GB | **Dask** | Distribuido, puede escalar |
| Análisis interactivo | **Pandas** | Mejor para exploración |
| Pipelines ETL | **Polars** | API clara, evaluación lazy |
| Computación distribuida | **Dask** | Paralelización automática |
| Necesidad de GPU | **Especial** | Considerar RAPIDS, CuDF |
| Compatibilidad máxima | **Pandas** | Más librerías lo soportan |

In [ ]:
# Comparativa de tiempos
import time

# Crear datos medianos
n = 5_000_000
datos = {
    'id': range(n),
    'valor': np.random.rand(n),
    'categoria': np.random.choice(['A', 'B', 'C', 'D'], n)
}

# Benchmark Pandas
start = time.time()
df_pandas = pd.DataFrame(datos)
resultado_pandas = df_pandas[df_pandas['valor'] > 0.5].groupby('categoria')['valor'].mean()
tiempo_pandas = time.time() - start

# Benchmark Polars
start = time.time()
df_polars = pl.DataFrame(datos)
resultado_polars = df_polars.filter(pl.col('valor') > 0.5).groupby('categoria').agg(pl.col('valor').mean())
tiempo_polars = time.time() - start

print(f"Pandas: {tiempo_pandas:.4f}s")
print(f"Polars: {tiempo_polars:.4f}s")
print(f"Polars es {tiempo_pandas/tiempo_polars:.1f}x más rápido")

## Casos de Uso Reales

### Caso 1: ETL de Datos Grandes

**Escenario:** Procesar archivos Parquet de 10GB diarios

**Solución Polars:**
```python
df = pl.scan_parquet('data/**/*.parquet')
result = (df
    .filter(pl.col('date') >= '2024-01-01')
    .groupby('category').agg(pl.col('amount').sum())
    .sort('amount', descending=True)
    .collect()
)
result.write_parquet('output.parquet')
```

### Caso 2: Dashboard en Vivo

**Escenario:** Datos que cambian constantemente, necesitas resp rápidas

**Solución Polars:**
- Carga datos en memoria con Polars
- Ejecuta consultas lazy para máxima velocidad
- Refrescar cada N segundos

### Caso 3: Análisis Exploratorio

**Escenario:** Exploración interactiva de datos

**Recomendación:** Pandas + Jupyter
- Mejor para análisis ad-hoc
- Más visualizaciones disponibles
- Transición a Polars si es lento

### Caso 4: Big Data Distribuido

**Escenario:** Cluster con múltiples máquinas, TB de datos

**Solución Dask:**
```python
ddf = dd.read_parquet('s3://bucket/data/')
result = ddf[ddf['value'] > 100].groupby('category').mean().compute()
```

## Resumen y Conclusiones

### Características Clave de Polars:

✓ **Velocidad:** 5-10x más rápido que Pandas

✓ **Memoria:** Mejor gestión de RAM

✓ **Lazy Evaluation:** Optimización automática

✓ **API moderna:** Expresiones composables

✓ **Tipos ricos:** Mejor manejo de datos complejos

### Cuando Usar Cada Herramienta:

| Herramienta | Mejor Para | No Usar Para |
| :--- | :--- | :--- |
| **Pandas** | Análisis interactivo, <1GB | Datos muy grandes |
| **Polars** | ETL, 1-20GB, velocidad | Cuando Pandas es suficiente |
| **Dask** | Big Data distribuido, >20GB | Datos pequeños (overhead) |

### Próximos Pasos:

1. Practica migrando código Pandas a Polars
2. Mide performance en tus datasets
3. Explora lazy evaluation
4. Aprende sobre optimizaciones específicas del dominio

¡Ya estás listo para usar Polars en producción!